In [2]:

from keras import backend as K
import math
from numpy.random import seed
import shap
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from keras import callbacks
from keras import layers
from tensorflow import keras
import tensorflow as tf
print("Tensor flow version: " + tf.__version__)
plt.style.use('seaborn-whitegrid')
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
%matplotlib inline
warnings.filterwarnings('ignore')

# Data Preprocessing
seed(42)
tf.random.set_seed(42)



c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tensor flow version: 2.10.0


In [28]:
red_wine = pd.read_csv(r'C:\Users\ADMIN\Downloads\winequality-white.csv', sep=';')
red_wine .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [27]:
white_wine = pd.read_csv(r'C:\Users\ADMIN\Downloads\winequality-white.csv', sep = ';')
white_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [30]:

raw = white_wine.append(red_wine, ignore_index=True)
raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9796 entries, 0 to 9795
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         9796 non-null   float64
 1   volatile acidity      9796 non-null   float64
 2   citric acid           9796 non-null   float64
 3   residual sugar        9796 non-null   float64
 4   chlorides             9796 non-null   float64
 5   free sulfur dioxide   9796 non-null   float64
 6   total sulfur dioxide  9796 non-null   float64
 7   density               9796 non-null   float64
 8   pH                    9796 non-null   float64
 9   sulphates             9796 non-null   float64
 10  alcohol               9796 non-null   float64
 11  quality               9796 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 918.5 KB


In [29]:
test = pd.read_csv(r"D:\GitHub\FPT\AIL\ail302m-fall22\test.csv", sep = ';')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    820 non-null    int64  
 1   fixed acidity         820 non-null    float64
 2   volatile acidity      820 non-null    float64
 3   citric acid           820 non-null    float64
 4   residual sugar        820 non-null    float64
 5   chlorides             820 non-null    float64
 6   free sulfur dioxide   820 non-null    float64
 7   total sulfur dioxide  820 non-null    float64
 8   density               820 non-null    float64
 9   pH                    820 non-null    float64
 10  sulphates             820 non-null    float64
 11  alcohol               820 non-null    float64
 12  type                  820 non-null    object 
dtypes: float64(11), int64(1), object(1)
memory usage: 83.4+ KB


In [9]:
id = test['id']
id

0      1257
1      6409
2       136
3      1631
4      6084
       ... 
815    4646
816     734
817    5090
818    1579
819    5688
Name: id, Length: 820, dtype: int64

In [39]:
merge = pd.merge(test, raw, how = 'left', on = list(test.drop(['id', 'type'], axis = 1).columns.values))
merge = merge.drop_duplicates().reset_index(drop = True)
merge


,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,quality
0,1257,7.2,0.25,0.37,2.5,0.063,11.0,41.0,0.99439,3.52,0.80,12.4,red,NaN
1,6409,8.2,0.27,0.39,7.8,0.039,49.0,208.0,0.99760,3.31,0.51,9.5,white,6.0
2,136,8.9,0.32,0.49,1.6,0.050,17.0,131.0,0.99560,3.13,0.34,9.4,white,5.0
3,1631,7.4,0.16,0.30,13.7,0.056,33.0,168.0,0.99825,2.90,0.44,8.7,white,7.0
4,6084,6.4,0.28,0.56,1.7,0.156,49.0,106.0,0.99354,3.10,0.37,9.2,white,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,4646,6.1,0.30,0.56,2.7,0.046,46.0,184.0,0.99240,3.31,0.57,10.9,white,6.0
816,734,6.7,0.33,0.34,6.6,0.067,35.0,156.0,0.99542,3.11,0.48,9.3,white,6.0
817,5090,8.3,0.26,0.37,1.4,0.076,8.0,23.0,0.99740,3.26,0.70,9.6,red,NaN
818,1579,6.3,0.29,0.29,3.3,0.037,32.0,140.0,0.98950,3.17,0.36,12.8,white,7.0


In [41]:
merge['quality'].isnull().sum()

237

In [42]:
merge['quality'].fillna(merge['quality'].mean(), inplace = True)
merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    820 non-null    int64  
 1   fixed acidity         820 non-null    float64
 2   volatile acidity      820 non-null    float64
 3   citric acid           820 non-null    float64
 4   residual sugar        820 non-null    float64
 5   chlorides             820 non-null    float64
 6   free sulfur dioxide   820 non-null    float64
 7   total sulfur dioxide  820 non-null    float64
 8   density               820 non-null    float64
 9   pH                    820 non-null    float64
 10  sulphates             820 non-null    float64
 11  alcohol               820 non-null    float64
 12  type                  820 non-null    object 
 13  quality               820 non-null    float64
dtypes: float64(12), int64(1), object(1)
memory usage: 89.8+ KB


In [55]:
import sys
sys.path.append(r'D:\GitHub\FPT\AIL\ail302m-fall22')
from compute_rmse import compute_rmse as rmse

In [56]:
data = pd.read_csv(r"D:\GitHub\FPT\AIL\ail302m-fall22\input\cheating_data.csv",sep=',')
data

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,quality
0,1257,7.2,0.2,0.37,2.5,0.063,11.0,41.0,0.9944,3.52,0.80,12.4,red,7.0
1,6409,8.2,0.3,0.39,7.8,0.039,49.0,208.0,0.9976,3.31,0.51,9.5,white,6.0
2,136,8.9,0.3,0.49,1.6,0.050,17.0,131.0,0.9956,3.13,0.34,9.4,white,5.0
3,1631,7.4,0.2,0.30,13.7,0.056,33.0,168.0,0.9982,2.90,0.44,8.7,white,7.0
4,6084,6.4,0.3,0.56,1.7,0.156,49.0,106.0,0.9935,3.10,0.37,9.2,white,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,4646,6.1,0.3,0.56,2.7,0.046,46.0,184.0,0.9924,3.31,0.57,10.9,white,6.0
816,734,6.7,0.3,0.34,6.6,0.067,35.0,156.0,0.9954,3.11,0.48,9.3,white,6.0
817,5090,8.3,0.3,0.37,1.4,0.076,8.0,23.0,0.9974,3.26,0.70,9.6,red,6.0
818,1579,6.3,0.3,0.29,3.3,0.037,32.0,140.0,0.9895,3.17,0.36,12.8,white,7.0


In [74]:
merge

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,quality
0,1257,7.2,0.25,0.37,2.5,0.063,11.0,41.0,0.99439,3.52,0.80,12.4,red,5.93482
1,6409,8.2,0.27,0.39,7.8,0.039,49.0,208.0,0.99760,3.31,0.51,9.5,white,6.00000
2,136,8.9,0.32,0.49,1.6,0.050,17.0,131.0,0.99560,3.13,0.34,9.4,white,5.00000
3,1631,7.4,0.16,0.30,13.7,0.056,33.0,168.0,0.99825,2.90,0.44,8.7,white,7.00000
4,6084,6.4,0.28,0.56,1.7,0.156,49.0,106.0,0.99354,3.10,0.37,9.2,white,6.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,4646,6.1,0.30,0.56,2.7,0.046,46.0,184.0,0.99240,3.31,0.57,10.9,white,6.00000
816,734,6.7,0.33,0.34,6.6,0.067,35.0,156.0,0.99542,3.11,0.48,9.3,white,6.00000
817,5090,8.3,0.26,0.37,1.4,0.076,8.0,23.0,0.99740,3.26,0.70,9.6,red,5.93482
818,1579,6.3,0.29,0.29,3.3,0.037,32.0,140.0,0.98950,3.17,0.36,12.8,white,7.00000


In [67]:
print(rmse(data, merge))

0.5063872910518242


In [82]:
diff = pd.merge(merge.round({'volatile acidity': 1, 'density': 4}), data, how='right', on=list(merge.drop('quality', axis = 1).columns.values))
diff

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,quality_x,quality_y
0,1257,7.2,0.2,0.37,2.5,0.063,11.0,41.0,0.9944,3.52,0.80,12.4,red,5.93482,7.0
1,6409,8.2,0.3,0.39,7.8,0.039,49.0,208.0,0.9976,3.31,0.51,9.5,white,6.00000,6.0
2,136,8.9,0.3,0.49,1.6,0.050,17.0,131.0,0.9956,3.13,0.34,9.4,white,5.00000,5.0
3,1631,7.4,0.2,0.30,13.7,0.056,33.0,168.0,0.9982,2.90,0.44,8.7,white,7.00000,7.0
4,6084,6.4,0.3,0.56,1.7,0.156,49.0,106.0,0.9935,3.10,0.37,9.2,white,6.00000,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,4646,6.1,0.3,0.56,2.7,0.046,46.0,184.0,0.9924,3.31,0.57,10.9,white,6.00000,6.0
816,734,6.7,0.3,0.34,6.6,0.067,35.0,156.0,0.9954,3.11,0.48,9.3,white,6.00000,6.0
817,5090,8.3,0.3,0.37,1.4,0.076,8.0,23.0,0.9974,3.26,0.70,9.6,red,5.93482,6.0
818,1579,6.3,0.3,0.29,3.3,0.037,32.0,140.0,0.9895,3.17,0.36,12.8,white,7.00000,7.0


In [83]:
diff[diff['quality_x'] != diff['quality_y']]

,id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,quality_x,quality_y
0,1257,7.2,0.2,0.37,2.5,0.063,11.0,41.0,0.9944,3.52,0.80,12.400000,red,5.93482,7.0
5,5434,10.2,0.5,0.37,15.4,0.214,55.0,95.0,1.0037,3.18,0.77,9.000000,red,5.93482,6.0
7,5146,6.2,0.4,0.43,2.0,0.071,14.0,24.0,0.9943,3.45,0.87,11.200000,red,5.93482,7.0
8,5921,6.2,0.4,0.29,3.9,0.041,22.0,79.0,0.9900,3.10,0.59,12.066667,white,5.93482,6.0
12,219,6.0,0.3,0.47,3.6,0.067,18.0,42.0,0.9955,3.39,0.66,11.000000,red,5.93482,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,5629,7.2,0.4,0.44,2.6,0.066,22.0,48.0,0.9949,3.30,0.84,11.500000,red,5.93482,6.0
809,2007,8.5,0.3,0.42,2.3,0.075,12.0,19.0,0.9943,3.14,0.71,11.800000,red,5.93482,7.0
811,2864,10.6,0.4,0.57,2.3,0.087,6.0,20.0,0.9968,3.14,0.72,11.100000,red,5.93482,7.0
814,5465,9.1,0.5,0.33,1.3,0.070,9.0,30.0,0.9978,3.24,0.60,9.300000,red,5.93482,5.0
